<a href="https://colab.research.google.com/github/fakelaboratory/publish/blob/main/ModernGL_triangle_animation_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install moderngl
!pip install moviepy
import moderngl as gl
import numpy as np
from PIL import Image
import moviepy.editor as mpy
from google.colab import files

ctx = gl.create_context(standalone=True, backend='egl')
print(ctx.info)

In [ ]:
prog = ctx.program(
    vertex_shader="""
        #version 330

        in vec2 in_vert;
        in vec3 in_color;

        out vec3 v_color;

        void main() {
            v_color = in_color;
            gl_Position = vec4(in_vert, 0.0, 1.0);
        }
    """,
    fragment_shader="""
        #version 330

        in vec3 v_color;

        out vec3 f_color;

        void main() {
            f_color = v_color;
        }
    """,
)

vertices = np.asarray([

    -0.75, -0.75,  1, 0, 0,
    0.75, -0.75,  0, 1, 0,
    0.0, 0.649,  0, 0, 1

], dtype='f4')

In [ ]:
def render_frame(time):
  vbo = ctx.buffer(vertices.tobytes())
  vao = ctx.vertex_array(prog, vbo, "in_vert", "in_color")

  fbo = ctx.framebuffer(
    color_attachments=[ctx.texture((512, 512), 3)]
  )
  fbo.use()
  fbo.clear(0.0+time, 0.0+time, 0.0+time, 1.0)
  vao.render()

  return np.array(Image.frombytes(
    "RGB", fbo.size, fbo.color_attachments[0].read(),
    "raw", "RGB", 0, -1
  ))

In [ ]:
clip = mpy.VideoClip(render_frame, duration=2) # 2 seconds
clip.write_gif("anim.gif",fps=15)
#now in files anim.gif